# Differential Cryptanalysis Tutorial

## What is Differential Cryptanalysis
Differential Cryptanalysis is a "chosen plaintext" attack. We choose plaintexts in order to find the secret key.

## Block Ciphers
Block ciphers operate on a fixed-length block of bits. When the message is split into blocks and encrypted/decrypted. Recall the Feistel Round we created earlier in the term. That used blocks of 64-bits. In today's tutorial we will be using basic 4-bit blocks.

Differential cryptanalysis can break some symmetric key block ciphers.

## XOR Review

Remember the carat (^) represents the XOR symbol in Python.

* 0 ^ 0 = 0
* 0 ^ 1 = 1
* 1 ^ 0 = 0
* 1 ^ 1 = 0

In [ ]:
# Remember the length of the key must match the length of the block to obtain a meaningful XOR result

block_bits = 0b01101110
key = 0b11010010

"""
Calculation by Hand:
 01101110
+11010010
 ________
 10111100
"""

bin(block_bits ^ key)

'0b10111100'

## Substitution

A step in which some bits are directly substituted for others.

In [ ]:
s_box = {
    0x0: 0xE, 0x1: 0x4, 0x2: 0xD, 0x3: 0x1,
    0x4: 0x2, 0x5: 0xF, 0x6: 0xB, 0x7: 0x8,
    0x8: 0x3, 0x9: 0xA, 0xA: 0x6, 0xB: 0xC,
    0xC: 0x5, 0xD: 0x9, 0xE: 0x0, 0xF: 0x7
}

"""
In this s_box, for example, the value of 0 is mapped to 0xE (15) and the value of 9 is mapped to 0xA (10).

This will be the s_box that we use for the rest of the demonstration.
"""

s_box[9]

10

## Toy Cipher

The toy chiper has a block size of 4 bits (input 4 bits of plaintext, output 4 bits of ciphertext). We also have the S-Box created above and two keys that we shall choose.

Encryption Process:
1. We XOR the plaintext with key1
2. Substitute that result using the S-box
3. XOR the output of the S-box with key2

![Alt Text](Process.png)

In [36]:
key_1 = 0xb
key_2 = 0xd

def toy_cipher(plaintext, s_box, key_1, key_2):
    """
    Python Implementation of a toy cipher

    Inputs: plaintext, s_box, key_1, key_2
    Output: ciphertext
    """
    # Step 1: XOR plaintext with key_1
    xored_plaintext = add_round_key(plaintext, int(key_1))
    # Step 2: Substitute the xored_plaintext with the s_box
    subsituted_xored_plaintext = substitute(xored_plaintext, s_box)
    # Step 3: XOR the substituted_xored_plaintext with key_2
    ciphertext = add_round_key(subsituted_xored_plaintext, int(key_2))

    return ciphertext

def substitute(state, s_box):
    return s_box[state]

def add_round_key(state, key):
    return state ^ key

toy_cipher(0b1011, s_box, key_1, key_2)

3

![Alt Text](Calculations.png)

## Differential Cryptanalysis of the Toy Cipher

Differential cryptanalysis analyzes the XOR of two plaintexts through the cipher. XOR tells us where two plaintexts differ. Thus, for every possible of input (0-15) we will find the corresponding number that when XOR'ed creates the same number, for example (11). 

Using the S-Box, we can transform the plaintext and analyze the output of the s_box xored values.

In [37]:
def analyze_xor(s_box, analyze_value):
    """
    Analyze the XOR operation of a specfic value. From all values 0-15 (x), find the corresponding value (x*) that XORs to the analyze_value. 
    Then, use the s_box on x and x* to find the corresponding y and y* values. Then XOR y and y* to find the values that are essential to cryptanalysis.

    Inputs: 
        s_box
        analyze_value (int): The value that we want to analyze the XOR operation for

    Outputs: 
        x_values, x_stars, xor_xx_stars, y_values, y_stars, xor_yy_stars
        These are all lists that contain the values of x, x*, x ^ x*, y, y*, and y ^ y* respectively.
    """
    x_values = []
    x_stars = []
    xor_xx_stars = []
    y_values = []
    y_stars = []
    xor_yy_stars = []

    for x in s_box.keys():
        # f"{x:04b}" is used to convert the integer to a 4-bit binary number
        x_bin = f"{x:04b}" 
        for x_star in range(16):
            # how we solve for x_star
            if x ^ x_star == analyze_value:
                x_star_bin = f"{x_star:04b}"
                break

        xor_xx_star_bin = f"{x ^ x_star:04b}"
        
        # y and y* values come from substituting x and x* into s_box
        y_bin = f"{s_box[x]:04b}"
        y_star_bin = f"{s_box[x_star]:04b}"
        
        xor_yy_star_bin = f"{s_box[x] ^ s_box[x_star]:04b}"
        
        # Append each value to corresponding list
        x_values.append(x_bin)
        x_stars.append(x_star_bin)
        xor_xx_stars.append(xor_xx_star_bin)
        y_values.append(y_bin)
        y_stars.append(y_star_bin)
        xor_yy_stars.append(xor_yy_star_bin)
    
    return x_values, x_stars, xor_xx_stars, y_values, y_stars, xor_yy_stars

### Creating a datatable that Visualizes the Code Above

In [38]:
import pandas as pd

analyze_value = 0b1011

x_values, x_stars, xor_xx_stars, y_values, y_stars, xor_yy_stars = analyze_xor(s_box, analyze_value)

df = pd.DataFrame({
    "x": x_values,
    "x*": x_stars,
    "x ⊕ x*": xor_xx_stars,
    "y": y_values,
    "y*": y_stars,
    "y ⊕ y*": xor_yy_stars
})

df

,x,x*,x ⊕ x*,y,y*,y ⊕ y*
0,0000,1011,1011,1110,1100,0010
1,0001,1010,1011,0100,0110,0010
2,0010,1001,1011,1101,1010,0111
3,0011,1000,1011,0001,0011,0010
4,0100,1111,1011,0010,0111,0101
5,0101,1110,1011,1111,0000,1111
6,0110,1101,1011,1011,1001,0010
7,0111,1100,1011,1000,0101,1101
8,1000,0011,1011,0011,0001,0010
9,1001,0010,1011,1010,1101,0111


Consider what this table is really saying. If we pick two random plaintexts which XOR to 1011, put each of them through the S-box, and then XOR those S-box outputs, 8 out of 16 times, they will XOR to 0010. They will XOR to 0111 2 out of 16 times. They will never XOR to 1010. This bias towards certain XORs is what makes differential cryptanalysis possible.


# Creating a difference distribution table

This table shows the number of appearances of the Output XOR given the input XOR. Basically, we are using the function analyze_value from above on all numbers 0-15 and creating a table with the results.

*** The whole point of this is to try to narrow down the possibilites for keys so that we can more easily determine what the keys are

In [ ]:
def difference_distribution_table(s_box):
    """
    Create a visual representation and an array that represents the difference distribution table, which is just the 
    analyze_xor function but for all possible values from 0 to 15.

    Inputs: s_box
    Outputs: df (visual representation), diff_dist_table (array form)
    """
    
    # Initialize a 16x16 array with all 0s
    diff_dist_table = [[0 for i in range(16)] for i in range(16)]
    for x_prime in range(16):
        for x in range(16):
            x_star = x ^ x_prime # x_prime is the input XOR value we want to analyze
            y = s_box[x]
            y_star = s_box[x_star]
            y_prime = y ^ y_star # y_prime is the output XOR value we want to analyze
            # add one to the corresponding cell in the array
            diff_dist_table[x_prime][y_prime] += 1
    
    df = pd.DataFrame(diff_dist_table, index=[f"{i}" for i in range(16)], columns=[f"{i}" for i in range(16)])
    return df, diff_dist_table

In [25]:
diff_dist_table = difference_distribution_table(s_box)[0]
diff_dist_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,0,0,0,2,0,2,4,0,4,2,0,0
2,0,0,0,2,0,6,2,2,0,2,0,0,0,0,2,0
3,0,0,2,0,2,0,0,0,0,4,2,0,2,0,0,4
4,0,0,0,2,0,0,6,0,0,2,0,4,2,0,0,0
5,0,4,0,0,0,2,2,0,0,0,4,0,2,0,0,2
6,0,0,0,4,0,4,0,0,0,0,0,0,2,2,2,2
7,0,0,2,2,2,0,2,0,0,2,2,0,0,0,0,4
8,0,0,0,0,0,0,2,2,0,0,0,4,0,4,2,2
9,0,2,0,0,2,0,0,4,2,0,2,2,2,0,0,0


### Key addition does not affect the outcome of XOR

Because XOR is associative, commutative, and a value XORED by itself is zero, and a value XORED by zero is itself, we can show the statement above.

* (x ^ k) ^ (x_star ^ k) 
* = (x ^ x_star) ^ k ^ k 
* = x ^ x_star ^ 0 
* = x ^ x_star

Hence, we do not have to worry about key addition in our cryptanalysis, only substitution



### Breaking Keys

1. Pick a differential characteristic (an input and output XOR pair)

2. Find a good pair (a pair of plaintexts that XOR to the input chosen above and whose associated ciphertexts XOR to the output chosen above)

In [26]:
# remember the keys that we chose were 0xb and 0xd
import random

diff_dist_table = difference_distribution_table(s_box)[1]

def differential_characteristic():
    """
    Find a differential characteristic pair that is not 0. Basically, look at the difference distribution table and find a pair of input and 
    output XOR that is not 0.

    Output: differential_characteristic
    """
    while True:
        # x_prime = random.randint(0, 15)
        # y_prime = random.randint(0, 15)
        x_prime = 5
        y_prime = 10

        if diff_dist_table[x_prime][y_prime] != 0:
            differential_characteristic = x_prime, y_prime
            return differential_characteristic

### Picking a differential characteristic Visualized

Looking at the differential distribution table, we should find a pair that correspond to a value greater than 0.

![Alt Text](differentialdistirbutiontable.png)

In [ ]:
input_XOR, output_XOR = differential_characteristic()
input_XOR, output_XOR

(5, 10)

Now we need to find a pair of plaintexts that XOR to 5 and whose associate ciphertexts XOR to 10. In real life, we would need to guess various sets of keys, but for the sake of simplicity let's luckily guess the real keys.

In [ ]:
def find_good_pair(input_XOR, output_XOR):
    """
    Find a good pair (i, j) that satisfies the input_XOR and output_XOR values.

    Inputs: input_XOR, output_XOR
    Outputs: i, j, i_encrypt, j_encrypt
    """

    # assume we luckily guess the keys
    key_1 = 11
    key_2 = 13

    for i in range(16):
        for j in range(16):
            i_encrypt = toy_cipher(i, s_box, key_1, key_2)
            j_encrypt = toy_cipher(j, s_box, key_1, key_2)
            if i ^ j == input_XOR and i_encrypt ^ j_encrypt == output_XOR:
                return i, j, i_encrypt, j_encrypt


In [29]:
i, j, i_encrypt, j_encrypt = find_good_pair(input_XOR, output_XOR)
good_pair = i, j
encrypted_pair = i_encrypt, j_encrypt
good_pair

(3, 6)

![Alt Text](image.png)

Remember, the key does not change the XOR. The question now is how do we find the "?'s"

We can reconstuct a table that corresponds to the analysis of the XOR's to 5, using code already written.

In [30]:
analyze_value = input_XOR

x_values, x_stars, xor_xx_stars, y_values, y_stars, xor_yy_stars = analyze_xor(s_box, analyze_value)

df = pd.DataFrame({
    "x": x_values,
    "x*": x_stars,
    "x ⊕ x*": xor_xx_stars,
    "y": y_values,
    "y*": y_stars,
    "y ⊕ y*": xor_yy_stars
})

df

,x,x*,x ⊕ x*,y,y*,y ⊕ y*
0,0000,0101,0101,1110,1111,0001
1,0001,0100,0101,0100,0010,0110
2,0010,0111,0101,1101,1000,0101
3,0011,0110,0101,0001,1011,1010
4,0100,0001,0101,0010,0100,0110
5,0101,0000,0101,1111,1110,0001
6,0110,0011,0101,1011,0001,1010
7,0111,0010,0101,1000,1101,0101
8,1000,1101,0101,0011,1001,1010
9,1001,1100,0101,1010,0101,1111


Now we look for the rows where x ^ x_star = i and y ^ y_star = j!!!!

Those are the possible inputs to an S-box that XOR to 0xi and whose corresponding S-box outputs XOR to 0xj!

In [31]:
rows_as_lists = df.values.tolist()

for row in rows_as_lists:
    print(row)

['0000', '0101', '0101', '1110', '1111', '0001']
['0001', '0100', '0101', '0100', '0010', '0110']
['0010', '0111', '0101', '1101', '1000', '0101']
['0011', '0110', '0101', '0001', '1011', '1010']
['0100', '0001', '0101', '0010', '0100', '0110']
['0101', '0000', '0101', '1111', '1110', '0001']
['0110', '0011', '0101', '1011', '0001', '1010']
['0111', '0010', '0101', '1000', '1101', '0101']
['1000', '1101', '0101', '0011', '1001', '1010']
['1001', '1100', '0101', '1010', '0101', '1111']
['1010', '1111', '0101', '0110', '0111', '0001']
['1011', '1110', '0101', '1100', '0000', '1100']
['1100', '1001', '0101', '0101', '1010', '1111']
['1101', '1000', '0101', '1001', '0011', '1010']
['1110', '1011', '0101', '0000', '1100', '1100']
['1111', '1010', '0101', '0111', '0110', '0001']


In [32]:
def possible_values_pairs(output_XOR, diff_dist_table):
    possible_values_pairs = []
    rows_as_lists = diff_dist_table

    for row in rows_as_lists:
        if int(row[5], 2) == output_XOR:
            possible_values_pairs.append((row[0], row[3]))

    return possible_values_pairs
    

In [33]:
possible_values_pairs = possible_values_pairs(output_XOR, rows_as_lists)
possible_values_pairs

[('0011', '0001'), ('0110', '1011'), ('1000', '0011'), ('1101', '1001')]

x ^ key_1 = u
v ^ key_2 = y

key_1 = x ^ u
key_2 = v ^ y

In [34]:
def possible_key_pairs(x, y, possible_values_pairs):
    x_r, x_l = x
    y_r, y_l = y
    possible_key_pairs = []
    print(possible_values_pairs)
    for i in range(0, 2):
        u_l, u_r = possible_values_pairs[i]
        v_l, v_r = possible_values_pairs[i+1]
        print(int(u_l, 2), u_r, v_l, v_r)
        print(x_l, x_r, y_l, y_r)
        temp = []
        if int(x_l) ^ int(u_l, 2) == int(x_r) ^ int(u_r, 2):
            key_1 = int(x_1) ^ int(u_l, 2)
            print(key_1)
        if int(v_l) ^ int(y_l) == int(v_r) ^ int(y_r):
            key_2 = int(v_l) ^ int(y_l)
            print(key_2)
        
        
    return possible_key_pairs

In [35]:
possible_key_pairs(good_pair, encrypted_pair, possible_values_pairs)

[('0011', '0001'), ('0110', '1011'), ('1000', '0011'), ('1101', '1001')]
3 0001 0110 1011
6 3 4 14
6 1011 1000 0011
6 3 4 14


[]

# 